In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `C:\Users\tge13\Documents\optimal_control_julia\lec_12\Project.toml`
    Updating registry at `C:\Users\tge13\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Installed PyPlot ────────────────────── v2.11.2
   Installed RobotDynamics ─────────────── v0.3.4
   Installed Ipopt ─────────────────────── v0.7.0
   Installed StaticArrays ──────────────── v1.9.3
   Installed MeshCat ───────────────────── v0.13.2
   Installed FunctionalCollections ─────── v0.5.0
   Installed Xorg_libX11_jll ───────────── v1.8.6+0
   Installed Hiccup ────────────────────── v0.2.2
   Installed Xorg_libXdmcp_jll ─────────── v1.1.4+0
   Installed HTTP ──────────────────────── v0.9.17
   Installed CodecBzip2 ────────────────── v0.7.2
   Installed RobotZoo ──────────────────── v0.2.3
   Installed ChangesOfVariables ────────── v0.1.8
   Installed RecipesBase ───────────────── v1.3.4
   Installed MathOptInterface ──────────── v0.9.22
   Ins

LoadError: Error building `MeshCat`: 

7-Zip 21.07 (x64) : Copyright (c) 1999-2021 Igor Pavlov : 2021-12-26

Scanning the drive for archives:
1 file, 366785 bytes (359 KiB)

Extracting archive: C:\Users\tge13\AppData\Local\Temp\jl_JrfBxn\meshcat.zip
--
Path = C:\Users\tge13\AppData\Local\Temp\jl_JrfBxn\meshcat.zip
Type = zip
Physical Size = 366785
Comment = 2355f01a8310314c1d7fc7e51bd01006549beaae

ERROR: Cannot create symbolic link : Ŭ���̾�Ʈ�� �ʿ��� ������ ������ ���� �ʽ��ϴ�. : C:\Users\tge13\AppData\Local\Temp\jl_JrfBxn\meshcat-2355f01a8310314c1d7fc7e51bd01006549beaae\test\main.js
ERROR: Cannot create symbolic link : Ŭ���̾�Ʈ�� �ʿ��� ������ ������ ���� �ʽ��ϴ�. : C:\Users\tge13\AppData\Local\Temp\jl_JrfBxn\meshcat-2355f01a8310314c1d7fc7e51bd01006549beaae\test\main.min.js

Sub items Errors: 2

Archives with Errors: 1

Sub items Errors: 2
ERROR: LoadError: failed process: Process(`'C:\Users\tge13\AppData\Local\Programs\Julia-1.6.7\bin\..\libexec\7z.exe' x 'C:\Users\tge13\AppData\Local\Te

In [6]:
import Pkg; Pkg.add("PyPlot"); Pkg.build("PyCall")

   Resolving package versions...
  No Changes to `C:\Users\tge13\Documents\optimal_control_julia\lec_12\Project.toml`
  No Changes to `C:\Users\tge13\Documents\optimal_control_julia\lec_12\Manifest.toml`
    Building Conda ─→ `C:\Users\tge13\.julia\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\51cab8e982c5b598eea9c8ceaced4b58d9dd37c9\build.log`
    Building PyCall → `C:\Users\tge13\.julia\scratchspaces\44cfe95a-1eb2-52ea-b672-e2afdf69b78f\9816a3826b0ebf49ab4926e2b18842ad8b5c8f04\build.log`


In [10]:
using LinearAlgebra
using PyPlot
using ForwardDiff
using RobotZoo
const RZ = RobotZoo
using RobotDynamics
using Ipopt
using MathOptInterface
const MOI = MathOptInterface;

In [11]:
#Pendulum Dynamics
a = RobotZoo.Acrobot()
h = 0.1 #10 Hz

0.1

In [12]:
Nx = 4     # number of state
Nu = 1     # number of controls
Tfinal = 5.0 # final time
Nt = Int(Tfinal/h)+1   # number of time steps
thist = Array(range(0,h*(Nt-1), step=h));

n_nlp = (Nx+Nu)*Nt # number of decision variables
m_nlp = Nx*(Nt+1)

208

In [13]:
function dircol_dynamics(x1,u1,x2,u2)
    #Hermite-Simpson integration with first-order hold on u
    f1 = RZ.dynamics(a, x1, u1)
    f2 = RZ.dynamics(a, x2, u2)
    xm = 0.5*(x1 + x2) + (h/8.0)*(f1 - f2)
    um = 0.5*(u1 + u2)
    fm = RZ.dynamics(a, xm, um)

    ẋm = (-3/(2.0*h))*(x1 - x2) - 0.25*(f1 + f2)
    return fm - ẋm
end

dircol_dynamics (generic function with 1 method)

In [14]:
# Cost weights
Q = Diagonal([1.0*ones(2); 1.0*ones(2)]);
R = 0.1;

In [15]:
function stage_cost(x,u)
    return 0.5*((x-xgoal)'*Q*(x-xgoal)) + 0.5*u'*R*u
end

stage_cost (generic function with 1 method)

In [16]:
function cost(ztraj)
    z = reshape(ztraj,Nx+Nu,Nt)
    J = 0.0
    for k = 1:Nt
        J += stage_cost(z[1:Nx,k],z[(Nx+1):(Nx+Nu),k])
    end
    return J
end

cost (generic function with 1 method)

In [17]:
function dynamics_constraint!(c,ztraj)
    d = reshape(c,Nx,Nt-1)
    z = reshape(ztraj,Nx+Nu,Nt)
    for k = 1:(Nt-1)
        x1 = z[1:Nx,k]
        u1 = z[(Nx+1):(Nx+Nu),k]
        x2 = z[1:Nx,k+1]
        u2 = z[(Nx+1):(Nx+Nu),k+1]
        d[:,k] = dircol_dynamics(x1,u1,x2,u2)
    end
    return nothing
end

dynamics_constraint! (generic function with 1 method)

In [18]:
function con!(c,ztraj)
    z = reshape(ztraj,Nx+Nu,Nt)
    c[1:Nx] .= (z[1:Nx,1] - x0)
    @views dynamics_constraint!(c[(Nx+1):(end-Nx)],ztraj)
    c[(end-Nx+1):end] .= (z[1:Nx,end] - xgoal)
end

con! (generic function with 1 method)

In [19]:
struct ProblemMOI <: MOI.AbstractNLPEvaluator
    n_nlp::Int
    m_nlp::Int
    idx_ineq
    obj_grad::Bool
    con_jac::Bool
    sparsity_jac
    sparsity_hess
    primal_bounds
    constraint_bounds
    hessian_lagrangian::Bool
end

function ProblemMOI(n_nlp,m_nlp;
        idx_ineq=(1:0),
        obj_grad=true,
        con_jac=true,
        sparsity_jac=sparsity_jacobian(n_nlp,m_nlp),
        sparsity_hess=sparsity_hessian(n_nlp,m_nlp),
        primal_bounds=primal_bounds(n_nlp),
        constraint_bounds=constraint_bounds(m_nlp,idx_ineq=idx_ineq),
        hessian_lagrangian=false)

    ProblemMOI(n_nlp,m_nlp,
        idx_ineq,
        obj_grad,
        con_jac,
        sparsity_jac,
        sparsity_hess,
        primal_bounds,
        constraint_bounds,
        hessian_lagrangian)
end

function primal_bounds(n)
    x_l = -Inf*ones(n)
    x_u = Inf*ones(n)
    return x_l, x_u
end

function constraint_bounds(m; idx_ineq=(1:0))
    c_l = zeros(m)
    c_l[idx_ineq] .= -Inf

    c_u = zeros(m)
    return c_l, c_u
end

function row_col!(row,col,r,c)
    for cc in c
        for rr in r
            push!(row,convert(Int,rr))
            push!(col,convert(Int,cc))
        end
    end
    return row, col
end

function sparsity_jacobian(n,m)

    row = []
    col = []

    r = 1:m
    c = 1:n

    row_col!(row,col,r,c)

    return collect(zip(row,col))
end

function sparsity_hessian(n,m)

    row = []
    col = []

    r = 1:m
    c = 1:n

    row_col!(row,col,r,c)

    return collect(zip(row,col))
end

function MOI.eval_objective(prob::MOI.AbstractNLPEvaluator, x)
    cost(x)
end

function MOI.eval_objective_gradient(prob::MOI.AbstractNLPEvaluator, grad_f, x)
    ForwardDiff.gradient!(grad_f,cost,x)
    return nothing
end

function MOI.eval_constraint(prob::MOI.AbstractNLPEvaluator,g,x)
    con!(g,x)
    return nothing
end

function MOI.eval_constraint_jacobian(prob::MOI.AbstractNLPEvaluator, jac, x)
    ForwardDiff.jacobian!(reshape(jac,prob.m_nlp,prob.n_nlp), con!, zeros(prob.m_nlp), x)
    return nothing
end

function MOI.features_available(prob::MOI.AbstractNLPEvaluator)
    return [:Grad, :Jac]
end

MOI.initialize(prob::MOI.AbstractNLPEvaluator, features) = nothing
MOI.jacobian_structure(prob::MOI.AbstractNLPEvaluator) = prob.sparsity_jac

function solve(x0,prob::MOI.AbstractNLPEvaluator;
        tol=1.0e-6,c_tol=1.0e-6,max_iter=10000)
    x_l, x_u = prob.primal_bounds
    c_l, c_u = prob.constraint_bounds

    nlp_bounds = MOI.NLPBoundsPair.(c_l,c_u)
    block_data = MOI.NLPBlockData(nlp_bounds,prob,true)

    solver = Ipopt.Optimizer()
    solver.options["max_iter"] = max_iter
    solver.options["tol"] = tol
    solver.options["constr_viol_tol"] = c_tol

    x = MOI.add_variables(solver,prob.n_nlp)

    for i = 1:prob.n_nlp
        MOI.add_constraint(solver, x[i], MOI.LessThan(x_u[i]))
        MOI.add_constraint(solver, x[i], MOI.GreaterThan(x_l[i]))
        MOI.set(solver, MOI.VariablePrimalStart(), x[i], x0[i])
    end

    # Solve the problem
    MOI.set(solver, MOI.NLPBlock(), block_data)
    MOI.set(solver, MOI.ObjectiveSense(), MOI.MIN_SENSE)
    MOI.optimize!(solver)

    # Get the solution
    res = MOI.get(solver, MOI.VariablePrimal(), x)

    return res
end

solve (generic function with 1 method)

In [20]:
#Initial and goal states
x0 = [-pi/2; 0; 0; 0]
xgoal = [pi/2; 0; 0; 0]

4-element Vector{Float64}:
 1.5707963267948966
 0.0
 0.0
 0.0

In [21]:
#Guess
xguess = kron(ones(Nt)', x0)
# xguess[1,:] .= xtraj[1,:] + 0.1*randn(Nt)
uguess = zeros(Nt)'
z0 = reshape([xguess; uguess],(Nx+Nu)*Nt,1); # 255×1 Matrix

In [ ]:
prob = ProblemMOI(n_nlp,m_nlp)

# solve
z_sol = solve(z0,prob)

# reshape
ztraj = reshape(z_sol,Nx+Nu,Nt)
xtraj = ztraj[1:Nx,:]
utraj = ztraj[(Nx+1):(Nx+Nu),:];

In [ ]:
plot(thist,xtraj[1,:])
plot(thist,xtraj[2,:])
plot(thist,xguess[1,:])

In [ ]:
plot(thist,xtraj[3,:])
plot(thist,xtraj[4,:])

In [ ]:
plot(thist,utraj[1,:])

In [ ]:
using TrajOptPlots
using MeshCat
using StaticArrays

vis = Visualizer()
render(vis)

In [ ]:
TrajOptPlots.set_mesh!(vis, a)

In [ ]:
X1 = [SVector{4}(x) for x in eachcol(xtraj)];
visualize!(vis, a, thist[end], X1)